# PySindy

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.integrate import solve_ivp
from sklearn.metrics import mean_squared_error

# Ignore integration and solver convergence warnings
import warnings
from scipy.integrate.odepack import ODEintWarning
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=ODEintWarning)
import pysindy as ps
from tqdm import tqdm
from support_functions import *
from scipy import signal
import matplotlib

## Load dataset

It generates the database concernig the evolution over time of Snowshoe Hare and Canadian lynx 

In [ ]:
X,t,dt = generate_populations_dataset()
t = np.arange(0,60,dt)

In [ ]:
dt_new = 0.1
t_new = np.arange(t[0],t[-1],dt_new)
X_new = interpolation(t ,X, t_new)
print(len(t_new))
t = t_new
X = X_new
dt = dt_new

plt.figure()
plt.plot(t,X[:,0])
plt.plot(t,X[:,1])
plt.grid()


<br>
<br>
<br>
<br>
<br>
<br>

## Fit Sindy Model 



In [ ]:
# SINDY model fitting
feature_names = ['x', 'y']
threshold = 1e-20
alpha = 0
n_models = 500
subset_size = X.shape[0] - int(X.shape[0] * 0.10)
max_iter = 1000
ensemble_optimizer = ps.SR3(threshold = threshold ,max_iter= max_iter)
model = ps.SINDy(feature_names=feature_names, optimizer=ensemble_optimizer, feature_library=ps.PolynomialLibrary(6,library_ensemble=True))
model.fit(X, t=dt, ensemble=True, n_models=n_models, n_subset=subset_size, quiet=False, n_candidates_to_drop=4)

model.print()

ensemble_coefs = np.asarray(model.coef_list)
mean_ensemble_coefs = np.mean(ensemble_coefs, axis=0)
median_ensemble_coefs = np.median(ensemble_coefs, axis=0)

In [ ]:
# set initial conditons
x0_test = X[0,:]

# set integrator: solve_ivp , odeint
integrator = 'solve_ivp'

# set integration  parameters
integrator_kws = {}

# Only for solve_ivp
# RK45 , RK23 , DOP853 , Radau , BDF , LSODA
if integrator == 'solve_ivp':
    integrator_kws['method'] = 'RK45'
    #integrator_kws['max_step'] = 1e20
    #integrator_kws['min_step'] = 1e15
else:
    print()
    #mxstep = 1e20
    #integrator_kws['hmax'] = 10
    #integrator_kws['hmin'] = 10
    
integrator_kws['rtol'] = 1e-8
integrator_kws['atol'] = 1e-8

dt_test = 1e-3
t_test = np.arange(0,58,dt_test)

# function to zero out any short-term unstable models 
stable_ensemble_coefs , stable_list = integration_metric(model,x0_test,t_test,np.asarray(ensemble_coefs), ensemble_optimizer,integrator,integrator_kws)

In [ ]:
stable_ensemble_coefs_1 = stable_ensemble_coefs[stable_list]
mean_ensemble_coefs = np.mean(stable_ensemble_coefs_1, axis=0)
median_ensemble_coefs = np.median(stable_ensemble_coefs_1, axis=0)

upper_limit_ensemble_coefs = np.percentile(stable_ensemble_coefs_1,0.5, axis=0)


<br>
<br>
<br>
<br>
<br>
<br>

## Plot the results

In [ ]:
x0_test =X[0,:]
ensemble_optimizer.coef_ = mean_ensemble_coefs
x_test_sim_meam_coeff = model.simulate(x0_test, t_test, integrator=integrator,integrator_kws = integrator_kws)


In [ ]:
X,t,dt = generate_populations_dataset()
t = np.arange(0,60,dt)

font = {'family' : 'normal',
        'size'   : 15}

matplotlib.rc('font', **font)
fig, ax = plt.subplots(2,1,figsize=(10,10))
ax[0].set_title('Snowshoe hare')
ax[0].plot(t+1845,X[:,0],'--b')
ax[0].plot(t_test+1845, x_test_sim_meam_coeff[:,0], color="b")
ax[0].set_ylabel('n° of animals',fontsize=15)
ax[0].legend(['Real data','SINDy'])
ax[0].grid()
ax[1].set_title('Canadian lynx')
ax[1].plot(t+1845,X[:,1],'--r')
ax[1].plot(t_test+1845, x_test_sim_meam_coeff[:,1], color="r") 
ax[1].set_xlabel('year',fontsize=15)
ax[1].set_ylabel('n° of animals',fontsize=15)
ax[1].legend(['Real data','SINDy'])
ax[1].grid()

plt.show()

In [ ]:
import matplotlib
font = {'family' : 'normal',
        'size'   : 15}

matplotlib.rc('font', **font)

fig, ax = plt.subplots(1,2,figsize=(20,5))
ax[0].set_title('Snowshoe hare')
ax[0].plot(t+1845,X[:,0],'--b')
ax[0].plot(t_test+1845, x_test_sim_meam_coeff[:,0], color="b")
ax[0].set_ylabel('n° of animals',fontsize=15)
ax[0].legend(['Real data','SINDy'])
ax[0].set_xlabel('year',fontsize=15)
ax[0].grid()
ax[1].set_title('Canadian lynx')
ax[1].plot(t+1845,X[:,1],'--r')
ax[1].plot(t_test+1845, x_test_sim_meam_coeff[:,1], color="r") 
ax[1].set_xlabel('year',fontsize=15)
ax[1].set_ylabel('n° of animals',fontsize=15)
ax[1].legend(['Real data','SINDy'])
ax[1].grid()

plt.show()

In [ ]:
#downsampling
resampled = np.zeros_like(X)
resampled[:,0] = signal.resample(x_test_sim_meam_coeff[:,0],30)
resampled[:,1] = signal.resample(x_test_sim_meam_coeff[:,1],30)

font = {'family' : 'normal',
        'size'   : 15}

matplotlib.rc('font', **font)

fig, ax = plt.subplots(1,2,figsize=(20,5))
ax[0].set_title('Snowshoe hare')
ax[0].plot(t+1845,abs(X[:,0]-resampled[:,0]),'--b')
ax[0].set_ylabel('abs(real-predicted)',fontsize=15)
ax[0].set_xlabel('year',fontsize=15)
ax[0].grid()
ax[1].set_title('Canadian lynx')
ax[1].plot(t+1845,abs(X[:,1]-resampled[:,1]),'--r')
ax[1].set_xlabel('year',fontsize=15)
ax[1].set_ylabel('abs(real-predicted)',fontsize=15)
ax[1].grid()

plt.show()

dist0 = np.linalg.norm(X[:,0]-resampled[:,0],2)
dist1 = np.linalg.norm(X[:,1]-resampled[:,1],2)
print('The L2 norm of the Snowshoe hare is:'+str(dist0))
print('The L2 norm of the Canadian lynx is:'+str(dist1))
